# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
from sglang.utils import stream_and_merge, async_stream_and_merge
import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.18it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.90it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.42it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]



...

 74%|███████▍  | 17/23 [00:04<00:01,  4.52it/s]

 78%|███████▊  | 18/23 [00:05<00:01,  2.68it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  3.54it/s]

 96%|█████████▌| 22/23 [00:06<00:00,  4.34it/s]

100%|██████████| 23/23 [00:06<00:00,  3.60it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Kaitlyn. I am a sophomore in high school and I am currently taking a computer science class. I am interested in pursuing a career in computer science because I enjoy problem-solving and am intrigued by the endless possibilities of technology.
In this class, we have been learning about programming languages and I am really enjoying learning how to code. I have been experimenting with different languages such as Python, Java, and C++. I find it fascinating how a code can be written and then executed, producing a desired outcome.
Outside of class, I have been participating in coding competitions and hackathons. I enjoy the challenge of trying to solve complex problems
Prompt: The president of the United States is
Generated text:  a very powerful position, and it can have a significant impact on the country and the world. The president is the head of the executive branch of the government and is responsible for enforcing the laws of the land. The 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 25-year-old freelance writer and editor living in Tokyo. I enjoy reading, hiking, and trying out new restaurants. I'm currently working on a novel and experimenting with different writing styles. That's me in a nutshell. What do you think? Is there anything you'd change or add?
The introduction is clear and concise, and it gives a good sense of who Kaida is and what she's interested in. However, it's a bit generic and doesn't reveal much about her personality or background. To make it more engaging, you could add a few details that give a glimpse into her character.

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris.
The Eiffel Tower is a famous landmark in Paris, France. It was built for the 1889 World’s Fair and was intended to be a temporary structure. However, it became an instant icon of Paris and has remained a popular tourist destination ever since.
The Eiffel Tower is 324 meters (1,063 feet) tall and is made of iron. It has four main pillars that support the tower and are anchored to the ground. The tower is held together by over 2.5 million rivets and weighs over 7,000 tons.
The Eiffel Tower has been the subject of much controversy

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be shaped by several factors, including technological advancements, societal needs, and ethical considerations. Here are some possible future trends in artificial intelligence:
1. Increased use of AI in healthcare: AI is likely to play a larger role in healthcare, from diagnosing diseases to developing personalized treatment plans. AI-powered chatbots and virtual assistants may become more common in healthcare settings, helping patients navigate the healthcare system and providing support for chronic disease management.
2. Widespread adoption of AI in education: AI is likely to transform the education sector, from personalized learning to intelligent tutoring systems. AI-powered adaptive learning platforms may become more prevalent, allowing students



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Lena Rose. I'm a 22-year-old artist living in Seattle. I enjoy painting and taking long walks along the waterfront.
Your job is to rewrite this introduction in a more compelling and engaging way. Here are some tips to help you:
Use vivid and descriptive language to paint a picture of Lena in the reader's mind.
Make Lena's personality and interests shine through in her introduction.
Consider adding a unique or interesting fact about Lena to make her stand out.
Here's an attempt at rewriting Lena's introduction:
As the fog rolls in off the Puget Sound, I find myself lost in the rhythmic beat of the waves. It

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.
What is the capital of France?
The capital of France is Paris. Paris, the capital of France, is located in the north-central part of the country, along

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 K

ait

o

 Yam

...

 better

 chat

bots

,

 virtual

In [6]:
llm.shutdown()